In [1]:
import ibis

ibis.options.interactive = True

In [2]:
con = ibis.get_backend()

In [3]:
import pandas as pd
from pathlib import Path

In [48]:
data_dir = Path().cwd().parent / "data" / "raw" / "fbref"

processed_path = data_dir.parent.parent / "processed" / "fbref"

In [5]:
results_files = list(data_dir.glob("**/match_stats/results.csv"))
lineup_files = list(data_dir.glob("**/match_stats/lineup.csv"))
report_files = list(data_dir.glob("**/match_stats/report.csv"))
shooting_match = list(data_dir.glob("**/match_stats/shooting.csv"))

In [6]:
season_stats = [
    "defense",
    "gca",
    "keepers",
    "keepers_adv",
    "misc",
    "passing",
    "passing_types",
    "playing_time",
    "possession",
    "shooting",
    "standard",
]

In [7]:
match_stats = [
    "defense",
    "keeper",
    "misc",
    "passing",
    "passing_types",
    "possession",
    "summary",
]

In [8]:
results_schema = {
    "Competition_Name": "string",
    "Gender": "string",
    "Country": "string",
    "Season_End_Year": "int64",
    "Tier": "string",
    "Round": "string",
    "Wk": "int64",
    "Day": "string",
    "Date": "int64",
    "Time": "time",
    "Home": "string",
    "HomeGoals": "int64",
    "Home_xG": "int64",
    "Away": "string",
    "AwayGoals": "int64",
    "Away_xG": "int64",
    "Attendance": "int64",
    "Venue": "string",
    "Referee": "string",
    "Notes": "string",
    "MatchURL": "string",
}
results = con.create_table("results", schema=results_schema, overwrite=True)

In [29]:
import polars as pl

df = pl.scan_csv(results_files).collect()

In [31]:
competition_mapper = {
    "Premier League": ("ENG", "M", "1st"),
    "EFL Championship": ("ENG", "M", "2nd"),
    "EFL League One": ("ENG", "M", "3rd"),
    "EFL League Two": ("ENG", "M", "4th"),
    "National League": ("ENG", "M", "5th"),
    "La Liga": ("ESP", "M", "1st"),
    "Spanish Segunda División": ("ESP", "M", "2nd"),
    "Fußball-Bundesliga": ("GER", "M", "1st"),
    "2. Fußball-Bundesliga": ("GER", "M", "2nd"),
    "3. Fußball-Liga": ("GER", "M", "3rd"),
    "Serie A": ("ITA", "M", "1st"),
    "Serie B": ("ITA", "M", "2nd"),
    "Ligue 1": ("FRA", "M", "1st"),
    "Ligue 2": ("FRA", "M", "2nd"),
    "Frauen-Bundesliga": ("GER", "F", "1st"),
    "Championnat de France de Football Féminin": ("FRA", "F", "1st"),
    "FA Women's Super League": ("ENG", "F", "1st"),
    "Liga F": ("ESP", "F", "1st"),
    "Premier League 2": ("ENG", "M", "1st"),
    "Premier League 2 — Division 2": ("ENG", "M", "2nd"),
    "A-Junioren Bundesliga": ("GER", "M", "1st"),
    "U19 DFB Youth League": ("GER", "M", "1st"),
    "B-Junioren Bundesliga": ("GER", "M", "1st"),
    "U17 DFB Youth League": ("GER", "M", "1st"),
    "Première Ligue": ("FRA", "M", "1st"),
}

name_mappings = {
    "A-Junioren Bundesliga": "U19 DFB Youth League",
    "B-Junioren Bundesliga": "U17 DFB Youth League",
    "Première Ligue": "Ligue 1",
    "Budesliga": "Bundesliga",
}

In [33]:
df = df.with_columns(
    Competition_Name=pl.col("Competition_Name").replace(name_mappings),
    Gender=pl.col("Competition_Name").replace(
        {k: v[1] for k, v in competition_mapper.items()}
    ),
    Country=pl.col("Competition_Name").replace(
        {k: v[0] for k, v in competition_mapper.items()}
    ),
    Tier=pl.col("Competition_Name").replace(
        {k: v[2] for k, v in competition_mapper.items()}
    ),
    Date=pl.col("Date").str.to_date(),
).with_columns(
    Season_End_Year=pl.when(pl.col("Date").dt.month().gt(6))
    .then(pl.col("Date").dt.year())
    .otherwise(pl.col("Date").dt.year() - 1)
)

In [40]:
data_dir

PosixPath('/home/jimmy/Code/FantasyFootball/data/raw/fbref')

PosixPath('/home/jimmy/Code/FantasyFootball/data/processed/fbref')

In [47]:
df.sort("Date", "Country", "Tier", "Round", "Wk").write_parquet(
    processed_path / "match_stats" / "results.parquet"
)

In [49]:
lineup = pl.scan_csv(lineup_files).collect()

FileNotFoundError: No such file or directory (os error 2): /home/jimmy/Code/FantasyFootball/data/raw/fbref/ESP/1st/2019/match_stats/lineup.csv

In [ ]:
def get_fbref_data(data_dir):
    files = data_dir.glob("*.csv")
    data = {}
    for file in files:
        data[file.stem] = pd.read_csv(file)
    return data

In [ ]:
big5_data = get_fbref_data(data_dir / "big5")

In [ ]:
big5_data["2020_M_misc_team"].memory_usage(deep=True).sum() / 1024**2

In [ ]:
team_stats = set([k.split("_")[2] + "_team" for k in big5_data.keys() if "team" in k])
player_stats = set(
    [k.split("_")[2] + "_player" for k in big5_data.keys() if "player" in k]
)

In [ ]:
def join_csvs(regex_pattern, directory):
    """Join all csvs in a directory that match a regex pattern in their filename"""
    csvs = sorted(list(Path(directory).rglob(regex_pattern)))
    if len(csvs) == 0:
        raise FileNotFoundError(
            f"No files found in {directory} that match {regex_pattern}"
        )
    df = pd.concat([pd.read_csv(csv) for csv in csvs])
    if "Unnamed: 0" in df.columns:
        df.drop("Unnamed: 0", axis=1, inplace=True)
    return df

In [ ]:
t = join_csvs("*misc_player.csv", data_dir / "big5")

In [ ]:
test1 = pd.read_csv(
    "/home/jimmy/Dropbox/Code/Python/FantasyFootball/data/FBRef/ENG/t1_2013_M_defense_match_stats_team.csv"
)

In [ ]:
test1

In [ ]:
test1.columns